# 2 jug problem with capacities x, y
## Formulation
### state
{x: u, y: v}
### actions
1. empty jug 1
2. empty jug 2
3. fill jug 1
4. fill jug 2
5. transfer from jug 1 to jug 2
6. transfer from jug 2 to jug 1
### successor function
D = {x: u, y: v} for all u in {0,1.,x} and v in {0,1,..y}
f : D -> D
state = {x: u, y: v}
1. f(state) = {x: 0, y: v}
2. f(state) = {x: u, y: 0}
3. f(state) = {x: x, y: v}
4. f(state) = {x: u, y: y}
5. f(state) = {x: 0, y: v}
6. 

### Solution type : state space search (Dijkstra's single source shortest path algorithm)

In [ ]:
x, y = 3, 2
cap_goal = 2
actions = [f"e{x}l", f"e{y}l", f"f{x}l", f"f{y}l", f"t{x}l{y}l", f"t{y}l{x}l"]
start = {x:0, y:0}

visited = [[False] * (y+1) for _ in range(x+1)] # visited[3+1][4+1]
visited[0][0] = True

cost_array = [[float("inf")] * (y+1) for _ in range(x+1)] # cost_array[3+1][4+1]
cost_array[0][0] = 0

solution = {0:[start]}

In [ ]:
def successor_function(state: dict, action):
    next = state.copy()
    if action == f"e{x}l":
        # empty x jug
        next[x] = 0
    elif action == f"e{y}l":
        # empty y jug
        next[y] = 0
    elif action == "f{x}l":
        # fill x jug
        next[x] = x
    elif action == f"f{y}l":
        # fill y jug
        next[y] = y
    elif action == f"t{x}l{y}l":
        # transfer from x jug to y jug
        space_left = y-state[y]
        if state[x] < space_left:
            next[y] += state[x]
            next[x] = 0
        else:
            next[x] -= space_left
            next[y] = y
    elif action == f"t{y}l{x}l":
        # transfer from y jug to x jug
        space_left = x-state[x]
        if state[y] < space_left:
            next[x] += state[y]
            next[y] = 0
        else:
            next[y] -= space_left
            next[x] = x    
    return next

In [ ]:
def get_cost(state):
    return cost_array[state[x]][state[y]]
def set_cost(state, cost):
    cost_array[state[x]][state[y]] = cost

In [ ]:
def comparator(state):
    return cost_array[state[x]][state[y]]

In [ ]:
# greedy BFS a.k.a Dijkstra's
queue = [start]
visited[start[x]][start[y]] = True
while queue:
    state = queue.pop(0)

    for action in actions:
        next_state = successor_function(state, action)

        if not visited[next_state[x]][next_state[y]]:
            queue.append(next_state)
            visited[next_state[x]][next_state[y]] = True

        next_cost = get_cost(state) + 1
        if next_cost < get_cost(next_state):
            set_cost(next_state, next_cost)
            solution[next_cost] = solution.get(next_cost, [])
            solution[next_cost].append(next_state)
        
    # fix min priority queue
    queue = sorted(queue, key=comparator)


In [797]:
# solution extraction
def first_goal():
    for cost in solution:
        for state in solution[cost]:
            if cap_goal in state.values():
                return cost, state

def is_neighbor(state1, state2):
    for action in actions:
        if successor_function(state1, action) == state2:
            return True
    return False

def first_solution():
    cost, state = first_goal()
    res = [state]
    cost -= 1

    while cost in solution:
        for next_state in solution[cost]:
            if is_neighbor(next_state, state):
                res.insert(0, next_state)
                state = next_state
                cost -= 1
    return res

for step in first_solution():
    print(step)

{3: 0, 2: 0}
{3: 0, 2: 2}
